In [ ]:
# BIP Project 6 - Karell, Antonio, Hanna
# adapted from from https://www.youtube.com/watch?v=ZE2DANLfBIs&ab_channel=NeuralNine

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install huggingface
!pip install huggingface_hub
!pip install transformers
!pip install datasets
#change text into a vector of features (term frequency, inverse document frequency)
#term frequency (how important/common is this word), IDF - how frequent is this word compared to other docs (how distinctive is it???)

from sklearn.svm import LinearSVC #an algorithm for method of classifying, maximizing the distance between classifying samples (linear support vector)
#other variants possible: K nearest neighbors, random forest... supposedly LinearSVC gave the best results

In [ ]:
from datasets import load_dataset

data = load_dataset("liar")
#print(data["test"]["label"])
#using the LIAR dataset - https://paperswithcode.com/dataset/liar
#data['fake'] = data["test"]["label"]  #using this model, it ended up being only 19% accuracy... perhaps because of having 4 gradations of truth?
#data['text'] = data["test"]["statement"]
#data = data.drop("label", axis=1) #relabeling fake to be a numerical value rather than real/fake


### So this is simplifying the labeling to be just binary.
print(data["test"]["label"])
labelarray = []

for i in range(len(data["test"]["label"])):
 if data["test"]["label"][i] == 3:
    labelarray.append(0)
 else:
    labelarray.append(1)
print(labelarray)

#print(data["test"]["label"])
#data['fake'] = data["test"]["label"].apply(lambda x: 0 if x == "3" else 1) #only 3 is true... 2 is mostly true, 1 4 5 are false
#print(data["fake"])
#print(data["test"]["label"])


In [ ]:
X, y = data['text'], data['fake'] #identifying text column as input and fake column as output to predict
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10) #reserves 80% of the data for training and 20% for checking the generated classifying rules afterwards


vectorizer = TfidfVectorizer(stop_words="english") #changing training data text into numerical features

X_train_vectorized = vectorizer.fit_transform(X_train)
#fit_transform method helps standardize it and transform it to be centered on the mean
X_test_vectorized = vectorizer.transform(X_test)
#no need to vectorize y variables because we already converted them into numerical values of 0 and 1

classifier = LinearSVC() #in the future, we could add grid search or randomized search to improve results
classifier.fit(X_train_vectorized, y_train) #now we run the classifier algorithm on these numericized values of X and Y
classifier.score(X_test_vectorized, y_test) #now that the model is trained, we check it against the test sample, to see accuracy %

#much worse performance for tweets, much worse for precisely labeling in gradations of 4... 19%

0.1867704280155642

In [ ]:
#now let's try this on another text that hasn't been classified yet, to predict if it's fake
gdown.download('https://drive.google.com/uc?id=18bU9lBfh_hrQHUfwzQwkjOVyN3VIQIPs', "unidentified.txt")
with open("unidentified.txt", "r", encoding="utf-8") as f:
  text = f.read()
vectorized_text = vectorizer.transform([text]) #have to vectorize it again, to get numerical values for the algorithm
#text is in the form of a collection when we do .read(), so make sure to put []

classifier.predict(vectorized_text)
#documentation for linearSVC says that .predict method returns likely class value of output
#so in this case, result array([1]) means that it classified it as fake news ... and later it turned out to be indeed fake

In [ ]:
#here is a potential modification for determining what % of a site is fake news (giving a rough idea of how reputable a site may be--more likely real news or more likely clickbait)
# Step 1 - Scrape a desired website (probably could enhance this with an API, but for now, using .csv from other scrapers... APIFY Smart Article Extractor)


gdown.download('https://drive.google.com/uc?id=1YJ1qWFmPpMQb1cBgCa5gJl0bm4ow4EGD', 'guardianarticles.csv')

Xdata = pd.read_csv('guardianarticles.csv')  # we can replace this dataset with other text
Xdata_vectorized = vectorizer.transform(Xdata)
#no need to vectorize y variables because we already converted them into numerical values of 0 and 1

print(Xdata_vectorized)  ## why is this showing just one row of data, when Xdata had 84 rows?

#X_vectorized = vectorizer.transform(data)
#print(X_vectorized)
#Y_predicted = classifier.predict(X_vectorized)
#for i in range(len(Y_predicted)):
#  print(Y_predicted[i])


#for i in range(len(X_vectorized)):
# print("X=%s, Predicted=%s" % (X_vectorized[i], Y_predicted[i]))